# Exercice 6.

**Lluís Mas & Radu Spaimoc**

This notebook contains the implented steps and the solution of this exercice.

### Problem 1.

Let’s consider a WEP cipher consisting on; 8 bits key length, 8 bits IV length, 8
bits CRC length being (x^8 + 1) the CRC polynomial. The PRNG is implemented as a 8 bits
shift register. We denote:
<ul>
    <li>PS(i) as the shift register status at iteration i</li>
    <li>PS(i)[j] as the j th bit of the shift register at iteration i. Consider P S(i)[0] as the most
left bit</li>
    <li>PO(i) as the output bit of the PNRG at iteration i</li>
</ul>

<img src="eq1.png" align="left">

In [1]:
def crc(msg, div, code='00000000'):
    """Cyclic Redundancy Check
    Generates an error detecting code based on an inputted message
    and divisor in the form of a polynomial representation.
    
    Arguments:
        msg: The input message of which to generate the output code.
        div: The divisor in polynomial form. For example, if the polynomial
            of x^3 + x + 1 is given, this should be represented as '1011' in
            the div argument.
        code: This is an option argument where a previously generated code may
            be passed in. This can be used to check validity. If the inputted
            code produces an outputted code of all zeros, then the message has
            no errors.
            
    Returns:
        An error-detecting code generated by the message and the given divisor.
    """
    # Append the code to the message. If no code is given, default to '00000000'
    msg = msg + code

    # Convert msg and div into list form for easier handling
    msg = list(msg)
    div = list(div)

    # Loop over every message bit (minus the appended code)
    for i in range(len(msg)-len(code)):
        # If that messsage bit is 1, perform modulo 2 multiplication
        if msg[i] == '1':
            for j in range(len(div)):
                # Perform modulo 2 multiplication on each index of the divisor
                msg[i+j] = str((int(msg[i+j])+int(div[j]))%2)
    
    # Output the last error-checking code portion of the message generated
    return ''.join(msg[-len(code):])

In [2]:
def wep_cipher(text, key, init_vector):
    """ Applies wep chipher algorithm
    
    Arguments:
        text: information to be ciphered or deciphered
        key: Key used
        init_vector: Init vector

    Returns:
        De/Ciphered text
    """
    prng = pseudo_random_noise_generators(key, init_vector, text)
    return xor(prng, text)

In [3]:
def pseudo_random_noise_generators(key, init_vector, text):
    """ Generates pseudo random noise
    
    Arguments:
        key  -- Key used
        init_vector  -- IV
        text -- information to be ciphered

    Returns: String of the output bits
    """

    po = []
    ps = []
    
    for i in range(0, len(text)):
        if i == 0:
            # PS(0) = init_vector (+) key
            ps = list(xor(init_vector, key))        
        else:
            prev_ps = ps.copy()
            for j in range(0, len(ps)):                
                if j == 0:
                    # PS(i)[0] = PS(i − 1)[4] (+) PS(i − 1)[7]
                    ps[j] = xor(prev_ps[4], prev_ps[7])
                else:
                    # PS(i)[j] = PS(i−1)[j−1], j>0
                    ps[j] = prev_ps[j-1]
                
        # PO[i] = PS(i)[4] (+) PS(i)[7]
        po.append(xor(ps[4], ps[7]))
        
    return "".join(po)


In [4]:
def xor(num1, num2):
    """ XOR two binary numbers.
    
    Arguments:
        num1: First binary number
        num2: Second binary number
        
    Return: XOR result
    """
    
    y = int(num1, 2) ^ int(num2,2)
    xor_result = bin(y)[2:].zfill(max(len(num1),len(num2)))
    return xor_result

In [5]:
def hex_to_bin(hexdecimal):
    """Convert hexadecimal to binary.
    
    Arguments: 
        hexadecimal: Hexadecimal number
    
    Return: Hexadecimal number converted into binary.
    """
    return bin(int(hexdecimal, 16))[2:]        

### 1.1

Probe that the ciphered data for the clear text **data 0x0123**, key=0x33 and IV=0x11 **is: 0x667E92**

In [6]:
text_data = "0x0123"
key = "0x33"
iv = "0x11"
exp_chiper = "0x667E92"

First we convert all the hexadecimal variables into binary.

In [7]:
binary_text_data = hex_to_bin(text_data).zfill(16)
binary_text_data

'0000000100100011'

In [8]:
binary_key = hex_to_bin(key).zfill(8)
binary_key

'00110011'

In [9]:
binary_iv = hex_to_bin(iv).zfill(8)    
binary_iv

'00010001'

In [10]:
binary_exp_cipher = hex_to_bin(exp_chiper).zfill(24)
binary_exp_cipher

'011001100111111010010010'

#### CRC Length = x^8 + 1

We know that the CRC length is:

In [11]:
crc_length = "100000001"

#### CRC Calculation

Then we compute the CRC code for the binary text data:

In [12]:
crc_code = crc(binary_text_data, crc_length)
crc_code

'00100010'

#### Chipher

Next, the text data is chipered using the previous obtained CRC code and the WEP chiper:

In [13]:
text_to_cipher = binary_text_data + crc_code

In [14]:
ciphered_text = wep_cipher(text_to_cipher, binary_key, binary_iv)

#### Validation

To probe the obtained ciphered text is compared with the binary expected value:

In [15]:
ciphered_text == binary_exp_cipher

True

The values are equal, so we proved that the text data **0x0123** ciphered is **0x667E92**.

### 1.2

Probe how deciphering with a wrong key (key=0x22) an error condition is reported

#### Validation with correct key

In [16]:
dec_text = wep_cipher(ciphered_text, binary_key, binary_iv)
print('CRC Correct:', crc(dec_text[:16], crc_length, crc_code) == '00000000')

CRC Correct: True


#### Validation with wrong key

In [17]:
dec_text = wep_cipher(ciphered_text, hex_to_bin("0x22").zfill(8), binary_iv)
print('CRC Correct:', crc(dec_text[:16], crc_length, crc_code) == '00000000')

CRC Correct: False


As we can see with the wrong key the obtained result from the crc method is different.

### Problem 2.

Let’s assume a CCMP with 8 bits length block. The counter blocks are computed
as follows:

<center>ctr(0) = key >> 2</center>
<center>ctr(i) = crt(i − 1) >> 2 ⊕ 0x53, i > 0</center>

where (>>) is a circular shift to the right and (⊕) is a XOR operation. Consider:

<ul>
    <li>key = 0x22</li>
    <li>information = 0x0001020304</li>
    <li>AES as a XOR operation</li>
</ul>

In [55]:
def CCMP_process(information, length_block, key, IV, decipher_mode):
    """
    Arguments:
        information -- information to be deciphered
        length_block  -- length of the information blocks
        key  -- Key used
        IV  -- Initialized to 0
        decipher_mode -- Indicates if cipher or decipher

    Returns:
        CTR_result -- information ciphered or deciphered
        generated_MIC -- MIC generated
    """

    # CTR
    CTR_result = CTR_process(information, key, length_block)

    # CBC
    if not decipher_mode: 
        # Calculate MIC with the information
        generated_MIC = generate_MIC(information, key, IV, length_block)
    else:
        # Calculate MIC with the information deciphered
        generated_MIC = generate_MIC(CTR_result, key, IV, length_block)

    return CTR_result, generated_MIC

In [56]:
def CTR_process(information, key, length_block):
    """
    Arguments:
        Information -- Information to be proecessed
        key -- key used

    Returns:
        A String of the CTR operations results
    """
    # Generate information blocks
    info_blocks = generate_information_blocks(information, length_block)

    # Calculate the differents ctr
    counter_blocks = generate_counter_blocks(info_blocks, key) 

    # CTR operation
    information_result = CTR_operation(info_blocks, counter_blocks, key)
    
    return "".join(information_result)

In [57]:
def generate_information_blocks(information, length_block): 
    """
    Split the information in blocks of a length of the length_block value
    
    Arguments:
        information -- Information to be splited
        length_block -- length of the blocks

    Returns:
        info_blocks = Array of blocks of information
    """

    info_blocks = []
    for index in range(0,len(information),length_block):
        info_blocks.append(information[index:index+length_block])
    return info_blocks

In [58]:
def generate_counter_blocks(info_blocks, key):
    """
    ctr(0) = key ≫ 2
    For each infornation block, in the first iteration, the ctr will be the key with a circular shift to the right of 2 values.
    
    ctr(i) = ctr(i−1)≫2 XOR 0x53, i>0
    For the others iterations the ctr will be the previous ctr with a circular shift to the right of 2 values and an xor of this result with "0x53".

    Arguments:
        info_blocks -- Blocks of information
        key -- key used

    Returns:
        counter_blocks = an Array of counter blocks calculated (ctr)
    """
    counter_blocks = []
    xor_key = hex_to_bin("0x53").zfill(8)

    for i in range(len(info_blocks)):

        if i == 0:
            # ctr(0) = key ≫ 2
            counter_blocks.append(shift(key, 2))

        else:
            # ctr(i) = ctr(i−1)≫2 XOR 0x53, i>0
            prev_shift = shift(counter_blocks[i-1], 2)
            counter_blocks.append(xor(prev_shift, xor_key))
             
    return counter_blocks

In [59]:
def shift(list, n=0):
    """Shift elements of a list by n steps"""
    return list[-n:]+list[:-n]

In [60]:
def CTR_operation(info_blocks, counter_blocks, key):
    """
    For each information block, calculate the AES(xor) between the key and their corresponding ctr 
    and then, calculate the xor between the obtained result and the block of information

    Arguments:
        info_blocks -- Blocks of information
        counter_blocks -- the differents ctr
        key -- key used

    Returns:
        information_result = an Array of the CTR operations results
    """

    information_result = []

    for i in range(len(counter_blocks)):
        AES_simulation = xor(counter_blocks[i], key)
        information_result.append(xor(AES_simulation, info_blocks[i]))
    
    return information_result

In [61]:
def generate_MIC(information, key, IV, length_block):
    """Generate MIC (Message Integrated Check)
        First iteration -> xor of the first block information with the IV, calculate AES of the result obtained with the key.
        Others iterations -> xor of the block information with the result acumulated, calculate AES of the result obtained with the key.
    """

    # Generate information blocks
    info_blocks = generate_information_blocks(information, length_block)

    # Initialize the result to IV
    result = IV
    for info_block in info_blocks:
        # xor of the last result with the block of information
        result = xor(info_block,result)

        # xor of the result obtained with the key (AES result with key)
        result = xor(result,key)   
    
    return result

### 2.1 
Probe that the ciphered information is **0xAA522FB151** and its corresponding MIC is **0x26**.
Check that the same schema also works as a decipher.

In [19]:
BLOCK_LENGTH = 8
key = "0x22"
information = "0x0001020304"
iv = "0x0"

expected_chipered_info = "0xAA522FB151"
expected_mic = "0x26"

In [20]:
binary_key = hex_to_bin(key).zfill(BLOCK_LENGTH)
binary_key

'00100010'

In [21]:
binary_information = hex_to_bin(information).zfill(BLOCK_LENGTH*5)
binary_information

'0000000000000001000000100000001100000100'

In [22]:
binary_iv = hex_to_bin(iv).zfill(BLOCK_LENGTH)
binary_iv

'00000000'

In [23]:
binary_expected_chipered_info = hex_to_bin(expected_chipered_info)
binary_expected_chipered_info

'1010101001010010001011111011000101010001'

In [24]:
binary_expected_mic = hex_to_bin(expected_mic).zfill(BLOCK_LENGTH)
binary_expected_mic

'00100110'

### Cipher

In [62]:
# Cipher the information
decipher_mode = False
ciphered_info, cipher_generated_MIC = CCMP_process(binary_information, 
                                                   BLOCK_LENGTH, 
                                                   binary_key, 
                                                   binary_iv, 
                                                   decipher_mode)    

### Cipher Validation

In [73]:
print("Chiper info", " generated: " + str(ciphered_info) + "\nExpected: \t\t" + binary_expected_chipered_info)
print("Equals:\t\t\t\t\t\t\t   ", str(ciphered_info) == binary_expected_chipered_info)

Chiper info  generated: 1010101001010010001011111011000101010001
Expected: 		1010101001010010001011111011000101010001
Equals:							    True


In [96]:
print("Chiper mic", "generated: " + str(cipher_generated_MIC) + "\nExpected:\t      " + binary_expected_mic)
print("Equals:\t\t\t ", str(cipher_generated_MIC) == binary_expected_mic)

Chiper mic generated: 00100110
Expected:	      00100110
Equals:			  True


### Decipher

In [97]:
decipher_mode = True
decipher_info, decipher_generated_MIC = CCMP_process(ciphered_info, 
                                                     BLOCK_LENGTH, 
                                                     binary_key, 
                                                     binary_iv, 
                                                     decipher_mode)

### Decipher Validation

In [101]:
print("Decipher info", " generated: " + str(decipher_info) + "\nExpected:\t\t  " + binary_information)
print("Equals:\t\t\t\t\t\t\t     ", str(decipher_generated_MIC) == cipher_generated_MIC)

Decipher info  generated: 0000000000000001000000100000001100000100
Expected:		  0000000000000001000000100000001100000100
Equals:							      True
